In [1]:
import numpy as np
from ipywidgets import widgets,Layout
from functools import partial

In [19]:
v_car_uc=[]
v_car_lc=[]
v_sbin=[]
v_car_uc.append(' ')
v_car_lc.append(' ')
nb_chars=27
nb_bits=np.log2(nb_chars)
if nb_bits%1==0:
    nb_bits=int(nb_bits)
else:
    nb_bits=int(nb_bits)+1
    
for i in range(nb_chars):
    current_char=chr(i+0x41)
    v_car_uc.append(current_char)
    
for i in range(nb_chars):
    current_char=chr(i+0x61)
    v_car_lc.append(current_char)
    
#print('v_car_uc=',v_car_uc)
#print('v_car_lc=',v_car_lc)

for i in range(nb_chars):
    v_sbin.append(np.binary_repr(i,nb_bits))
#print('v_sbin=',v_sbin)

In [20]:
w_html=widgets.HTML('',layout=Layout(height='1000px',width='300px'))
w_text=widgets.Text(value='', placeholder='', description='', disabled=False,layout=Layout(width='300px'))
w_rb=widgets.RadioButtons(options=[('Alphabet -> Binaire',0), ('Binaire->Alphabet',1), ('Generation de masque',2)],description='',disabled=False)
def format_convert(string_in,bool_direction):
    nb_string_in=len(string_in)
    if bool_direction==0:
        s_out=''
        continueloop=True
        if nb_string_in==0:
            continueloop=False
            s_out=""
        i=0
        while(continueloop):
            current_str=string_in[i]
            is_current_str_exist_in_uc=bool(v_car_uc.count(current_str))
            is_current_str_exist_in_lc=bool(v_car_lc.count(current_str))
            is_current_str_exist=is_current_str_exist_in_lc | is_current_str_exist_in_uc
            if (is_current_str_exist):
                if is_current_str_exist_in_uc:
                    v_carac=v_car_uc
                else:
                    v_carac=v_car_lc
                    
                where_is_current=v_carac.index(current_str)
                s_out='%s%s'%(s_out,v_sbin[where_is_current])
                i+=1

                if (i<nb_string_in):
                    continueloop=True
                else:
                    continueloop=False                
            else:
                continueloop=False
                s_out="le caractères %s est inconnu"%current_str
    elif bool_direction==1:
        nb_char_to_be_translates=int(nb_string_in/nb_bits)
        s_out=''
        continueloop=True
        if nb_char_to_be_translates==0:
            continueloop=False
            s_out="la chaine binaire fait moins de %d bits"%(nb_bits)
            
        is_all_bin=True
        for j in range(nb_string_in):
            current_str=string_in[j]
            is_all_bin&=((current_str=='1')|(current_str=='0'))
            
        if (is_all_bin ==  False)& continueloop:
            s_out="la chaine n'est pas binaire"
            continueloop=False
            
        if ((nb_string_in%nb_bits)!=0) & continueloop:
            s_out="la longueur de la chaine binaire n'est pas un multiple de %d"%nb_bits
            continueloop=False
        i=0
        while(continueloop):
            current_str=''
            for j in range(nb_bits):
                current_str='%s%s'%(current_str,string_in[nb_bits*i+j])
            is_current_str_exist=bool(v_sbin.count(current_str))
            if is_current_str_exist:
                where_is_current=v_sbin.index(current_str)
                s_out='%s%s'%(s_out,v_car_uc[where_is_current])
                i+=1
                if (i<nb_char_to_be_translates):
                    continueloop=True
                else:
                    continueloop=False                   
            else:
                continueloop=False
                s_out="le chaine binaire %s ne correspond à aucune lettre"%current_str
    elif bool_direction==2:
        try:
            x = int(string_in)
            nb_bits_to_encrypt=x*nb_bits
            nb_bytes_to_encrypt=(nb_bits_to_encrypt/8)
            if nb_bytes_to_encrypt%1==0:
                nb_bytes_to_encrypt=int(nb_bytes_to_encrypt)
            else:
                nb_bytes_to_encrypt=int(nb_bytes_to_encrypt)+1
            v_rnd=np.random.randint(0,256,nb_bytes_to_encrypt,dtype=np.uint8)
            s_out=''
            for i in range(nb_bytes_to_encrypt):
                s_out='%s%.2x'%(s_out,v_rnd[i])
        except ValueError: 
            s_out="veuillez entrer le nombre de lettres de votre message"
    w_html.value='<p>%s</p>'%s_out
def change_on_param(obj):
    if obj['new']:
        format_convert(w_text.value,w_rb.value)
            
w_text.observe(partial(change_on_param),'value')
w_rb.observe(partial(change_on_param),'value')
v_box_top=widgets.VBox([w_text,w_rb,w_html])
display(v_box_top)            